# Comparison Models Training Notebook

This notebook trains all 8 comparison models for benchmarking against FUME-FastSCNN.

**Models:** BiSeNetV2, CMX, DDRNetSlim, RTFNet, ESPNetV2, MTINet, ENet, DANet

**Fast Training Configuration:**
- 15 epochs (instead of 50)
- Batch size: 16 (increased from 8)
- Learning rate: 0.002 (2x higher for fast convergence)
- 8 workers for faster data loading
- Minimal augmentation
- Mixed precision (AMP) enabled


## 1. Setup


In [1]:
import sys
import os
sys.path.append('..')

import torch
import yaml
from pathlib import Path

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")


PyTorch version: 2.8.0+cu128
CUDA available: True
CUDA device: NVIDIA RTX 6000 Ada Generation
CUDA version: 12.8


## 2. Configuration


In [2]:
config_path = '../configs/fast_comparison_config.yaml'

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

print("Fast Training Configuration:")
print(f"  Epochs: {config['training']['num_epochs']}")
print(f"  Batch size: {config['training']['batch_size']}")
print(f"  Learning rate: {config['training']['optimizer']['lr']}")
print(f"  Num workers: {config['training']['num_workers']}")
print(f"  Mixed precision: {config['training']['use_amp']}")
print(f"  W&B enabled: {config['experiment']['use_wandb']}")


Fast Training Configuration:
  Epochs: 15
  Batch size: 16
  Learning rate: 0.002
  Num workers: 8
  Mixed precision: True
  W&B enabled: True


## 3. Comparison Models Overview


In [3]:
from models import (
    BiSeNetV2,
    CMX,
    DDRNetSlim,
    RTFNet,
    ESPNetV2,
    MTINet,
    ENet,
    DANet
)

models_info = [
    {'name': 'BiSeNetV2', 'params': '3.4M', 'type': 'Bilateral dual-path'},
    {'name': 'CMX', 'params': '3.8M', 'type': 'Cross-modal transformer'},
    {'name': 'DDRNetSlim', 'params': '5.7M', 'type': 'Dual-resolution network'},
    {'name': 'RTFNet', 'params': '4.2M', 'type': 'RGB-Thermal fusion'},
    {'name': 'ESPNetV2', 'params': '1.2M', 'type': 'Spatial pyramid'},
    {'name': 'MTINet', 'params': '3.5M', 'type': 'Multi-task interaction'},
    {'name': 'ENet', 'params': '0.4M', 'type': 'Ultra-lightweight'},
    {'name': 'DANet', 'params': '3.2M', 'type': 'Dual attention'}
]

print("Models to train:")
print("="*70)
for i, model_info in enumerate(models_info, 1):
    print(f"{i}. {model_info['name']:<15} | {model_info['params']:<8} | {model_info['type']}")
print("="*70)


Models to train:
1. BiSeNetV2       | 3.4M     | Bilateral dual-path
2. CMX             | 3.8M     | Cross-modal transformer
3. DDRNetSlim      | 5.7M     | Dual-resolution network
4. RTFNet          | 4.2M     | RGB-Thermal fusion
5. ESPNetV2        | 1.2M     | Spatial pyramid
6. MTINet          | 3.5M     | Multi-task interaction
7. ENet            | 0.4M     | Ultra-lightweight
8. DANet           | 3.2M     | Dual attention


## 4. Verify All Models (Forward Pass Test)

Testing each model to ensure they work correctly before training.


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Testing on device: {device}\n")

MODEL_CLASSES = {
    'BiSeNetV2': BiSeNetV2,
    'CMX': CMX,
    'DDRNetSlim': DDRNetSlim,
    'RTFNet': RTFNet,
    'ESPNetV2': ESPNetV2,
    'MTINet': MTINet,
    'ENet': ENet,
    'DANet': DANet
}

dummy_co2 = torch.randn(2, 1, 480, 640).to(device)
dummy_ch4 = torch.randn(2, 1, 480, 640).to(device)
dummy_mask = torch.ones(2, 2).to(device)

print("="*70)
print("MODEL VERIFICATION TEST")
print("="*70)
print(f"{'Model':<15} {'Params':<12} {'Forward':<10} {'Output Shapes'}")
print("-"*70)

all_passed = True
for name, ModelClass in MODEL_CLASSES.items():
    try:
        model = ModelClass(num_classes=3, num_seg_classes=3).to(device)
        model.eval()
        
        num_params = model.get_num_parameters()
        
        with torch.no_grad():
            outputs = model(dummy_co2, dummy_ch4, dummy_mask)
        
        cls_shape = outputs['cls_logits'].shape
        seg_shape = outputs['co2_seg_logits'].shape
        
        status = "PASS"
        shapes = f"cls:{list(cls_shape)}, seg:{list(seg_shape)}"
        print(f"{name:<15} {num_params/1e6:.2f}M{'':<6} {status:<10} {shapes}")
        
        del model
        torch.cuda.empty_cache()
        
    except Exception as e:
        status = "FAIL"
        all_passed = False
        print(f"{name:<15} {'--':<12} {status:<10} Error: {str(e)[:40]}")

print("="*70)
if all_passed:
    print("All models passed verification! Ready to train.")
else:
    print("Some models failed. Please fix issues before training.")


Testing on device: cuda

MODEL VERIFICATION TEST
Model           Params       Forward    Output Shapes
----------------------------------------------------------------------
BiSeNetV2       2.77M       PASS       cls:[2, 3], seg:[2, 3, 480, 640]
CMX             4.41M       PASS       cls:[2, 3], seg:[2, 3, 480, 640]
DDRNetSlim      3.39M       PASS       cls:[2, 3], seg:[2, 3, 480, 640]
RTFNet          35.01M       PASS       cls:[2, 3], seg:[2, 3, 480, 640]
ESPNetV2        2.22M       PASS       cls:[2, 3], seg:[2, 3, 480, 640]
MTINet          1.80M       PASS       cls:[2, 3], seg:[2, 3, 480, 640]
ENet            0.26M       PASS       cls:[2, 3], seg:[2, 3, 480, 640]
DANet           1.43M       PASS       cls:[2, 3], seg:[2, 3, 480, 640]
All models passed verification! Ready to train.


## 5. Check Data Files


In [5]:
required_files = [
    '../data/paired_train_annotations.csv',
    '../data/paired_val_annotations.csv',
    '../data/paired_test_annotations.csv'
]

all_exist = all(os.path.exists(f) for f in required_files)

if all_exist:
    print("All paired annotation files found")
else:
    print("Paired annotation files not found. Run data/pairing.py first")
    print("\nRun this command in terminal:")
    print("  cd data && python pairing.py")


All paired annotation files found


## 6. Start Training All Models

This will train all 8 comparison models sequentially.


In [ ]:
os.chdir('..')
print(f"Working directory: {os.getcwd()}")

from train_all_comparison_models import FastTrainer

models_to_train = [
    'BiSeNetV2',
    'CMX',
    'DDRNetSlim',
    'RTFNet',
    'ESPNetV2',
    'MTINet',
    'ENet',
    'DANet'
]

results = []

print("\n" + "="*70)
print("BATCH TRAINING: COMPARISON MODELS")
print("="*70)
print(f"Config: configs/fast_comparison_config.yaml")
print(f"Models: {', '.join(models_to_train)}")
print(f"Total models: {len(models_to_train)}")
print("="*70 + "\n")

for i, model_name in enumerate(models_to_train, 1):
    print(f"\n{'#'*70}")
    print(f"# Training Model {i}/{len(models_to_train)}: {model_name}")
    print(f"{'#'*70}\n")
    
    try:
        trainer = FastTrainer('configs/fast_comparison_config.yaml', model_name)
        result = trainer.train()
        results.append(result)
        
        torch.cuda.empty_cache()
        
        print(f"\n{'='*70}")
        print(f"{model_name} Training Completed")
        print(f"Best Balanced Accuracy: {result['best_metric']:.4f}")
        print(f"Training Time: {result['training_time']/60:.2f} minutes")
        print(f"{'='*70}\n")
        
    except Exception as e:
        print(f"\nError training {model_name}: {e}")
        import traceback
        traceback.print_exc()
        continue

print("\n" + "="*70)
print("ALL MODELS TRAINING COMPLETED")
print("="*70)


Working directory: /home/siu856569517/Taminul/Acidosis/FUME


/home/siu856569517/.conda/envs/fume_env/lib/python3.9/site-packages/wandb/sdk/launch/builder/build.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.



BATCH TRAINING: COMPARISON MODELS
Config: configs/fast_comparison_config.yaml
Models: BiSeNetV2, CMX, DDRNetSlim, RTFNet, ESPNetV2, MTINet, ENet, DANet
Total models: 8


######################################################################
# Training Model 1/8: BiSeNetV2
######################################################################


🚀 Training BiSeNetV2
Device: cuda


/home/siu856569517/.conda/envs/fume_env/lib/python3.9/site-packages/wandb/sdk/launch/builder/build.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
wandb: Currently logged in as: taminul. Use `wandb login --relogin` to force relogin


INFO:data.dataset:Loaded 4383 paired samples
INFO:data.dataset:  Fully paired: 1893
INFO:data.dataset:  Modality dropout: 0.1
INFO:data.dataset:  Class distribution: {'Acidotic': 2734, 'Healthy': 1488, 'Transitional': 161}
INFO:data.dataset:Loaded 939 paired samples
INFO:data.dataset:  Fully paired: 406
INFO:data.dataset:  Modality dropout: 0.0
INFO:data.dataset:  Class distribution: {'Acidotic': 586, 'Healthy': 318, 'Transitional': 35}
/home/siu856569517/Taminul/Acidosis/FUME/train_all_comparison_models.py:259: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler() if self.config['training']['use_amp'] else None


...

Epoch 12/15:   0%|          | 0/274 [00:00<?, ?it/s]/home/siu856569517/Taminul/Acidosis/FUME/train_all_comparison_models.py:282: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.config['training']['use_amp']):
Epoch 12/15: 100%|██████████| 274/274 [02:45<00:00,  1.66it/s, loss=nan]



Epoch 12/15
  Train Loss: nan | Val Loss: nan
  Balanced Acc: 0.3333 | mIoU: 0.2674
  Best Balanced Acc: 0.3333
  Time: 2.92 min
----------------------------------------------------------------------


Epoch 13/15:   0%|          | 0/274 [00:00<?, ?it/s]/home/siu856569517/Taminul/Acidosis/FUME/train_all_comparison_models.py:282: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.config['training']['use_amp']):
Epoch 13/15: 100%|██████████| 274/274 [02:59<00:00,  1.53it/s, loss=nan]



Epoch 13/15
  Train Loss: nan | Val Loss: nan
  Balanced Acc: 0.3333 | mIoU: 0.2674
  Best Balanced Acc: 0.3333
  Time: 3.16 min
----------------------------------------------------------------------


Epoch 14/15:   0%|          | 0/274 [00:00<?, ?it/s]/home/siu856569517/Taminul/Acidosis/FUME/train_all_comparison_models.py:282: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.config['training']['use_amp']):
Epoch 14/15:  15%|█▍        | 41/274 [00:26<01:26,  2.68it/s, loss=nan]

## 7. Training Summary


In [ ]:
if results:
    print("\n" + "="*70)
    print("TRAINING SUMMARY - RANKED BY PERFORMANCE")
    print("="*70)
    
    results_sorted = sorted(results, key=lambda x: x['best_metric'], reverse=True)
    
    print(f"\n{'Rank':<6}{'Model':<15}{'Bal. Acc':<12}{'mIoU':<10}{'F1':<10}{'Time (min)'}")
    print("-"*70)
    
    for i, result in enumerate(results_sorted, 1):
        print(f"{i:<6}{result['model_name']:<15}"
              f"{result['best_metric']:<12.4f}"
              f"{result['final_results']['seg']['mean_iou']:<10.4f}"
              f"{result['final_results']['cls']['macro_f1']:<10.4f}"
              f"{result['training_time']/60:.2f}")
    
    print("="*70)
    
    print("\nDetailed Results:")
    print("="*70)
    for i, result in enumerate(results_sorted, 1):
        print(f"\n{i}. {result['model_name']}")
        print(f"   Best Balanced Accuracy: {result['best_metric']:.4f}")
        print(f"   Mean IoU: {result['final_results']['seg']['mean_iou']:.4f}")
        print(f"   Mean Dice: {result['final_results']['seg']['mean_dice']:.4f}")
        print(f"   Macro F1: {result['final_results']['cls']['macro_f1']:.4f}")
        print(f"   Training Time: {result['training_time']/60:.2f} min")
    
    print("\n" + "="*70)
    print("All models trained successfully")
    print("="*70)
else:
    print("No results available. Training may have failed.")


## 8. Save Results Summary


In [ ]:
if results:
    import json
    from datetime import datetime
    
    results_dir = Path('../results_comparison')
    results_dir.mkdir(exist_ok=True)
    
    summary = {
        'timestamp': datetime.now().isoformat(),
        'config': {
            'epochs': config['training']['num_epochs'],
            'batch_size': config['training']['batch_size'],
            'learning_rate': config['training']['optimizer']['lr'],
        },
        'results': results
    }
    
    summary_path = results_dir / 'training_summary.json'
    with open(summary_path, 'w') as f:
        json.dump(summary, f, indent=2)
    
    print(f"Results summary saved to: {summary_path}")


## 9. Check Model Checkpoints


In [ ]:
checkpoints_dir = Path('../checkpoints_comparison')

if checkpoints_dir.exists():
    print("Model Checkpoints:")
    print("="*70)
    
    for model_name in models_to_train:
        model_dir = checkpoints_dir / model_name
        if model_dir.exists():
            best_ckpt = model_dir / 'best_model.pth'
            last_ckpt = model_dir / 'last_model.pth'
            
            status = "✓" if best_ckpt.exists() else "✗"
            print(f"{status} {model_name:<15} - {model_dir}")
    
    print("="*70)
else:
    print("No checkpoints directory found.")


## Next Steps

1. Training completed
2. Evaluate all models on test set
3. Compare with FUME-FastSCNN results
4. Generate comparison tables and figures
5. Analyze performance differences
